# Exercise solutions: reading and writing files

## Exercise 3.1
Write a function that reads the first 10 lines of the file `Homo_sapiens.GRCh38.99.MT.gtf` and writes them to another file.

In [ ]:
with open("../exercises/Homo_sapiens.GRCh38.99.MT.gtf", 'r') as in_file, open("Homo_sapiens.GRCh38.99.MT.head.gtf", 'w') as out_file:
    for i in range(10):
        print(in_file.readline(), file=out_file, end='')


Notes :
* to open the input file we specify its location relatively to our notebook. Since the file is located in a different directory, we must use `../exercises` to reach it.
* We use the `end` optional argument of `print()` to avoid printing an additional `\n` character that would cause line skips (because each line already ends with a `\n` character read from the input).  
  Alternatively we could have used `strip()` on the lines read from the input: `print(in_file.readline().strip(), file=out_file)`
  

<br>

## Exercise 3.2
Using the same `Homo_sapiens.GRCh38.99.MT.gtf` GFF file you used in the previous exercise, report (print to terminal) all of the feature entries for genes on the motichondrial chromosome (`MT`) between coordinates 7500 and 10000 on the forward strand.  
The complete GFF file format description can be found <a href="https://www.ensembl.org/info/website/upload/gff.html">here</a>, but here is a succint description: 
* Column 1: chromosome
* Column 2: data source
* Column 3: feature type
* Column 4: feature start coordinate
* Column 5: feature end coordinate
* Column 6: score
* Column 7: strand

> hint: use the `split()` method; what is the separator between fields ?



In [ ]:
# In the code below, we will not presume that the features are ordered by chromosome and position, 
# and therefore we will read through the entire file.
# However, if that assumption can be made, then it could be good way to terminate the reading early, 
# and save some execution time if the input file is very big (note that the gtf file of the full 
# human genome is ~2 GB)

to_report = []
with open("../exercises/Homo_sapiens.GRCh38.99.MT.gtf" , 'r') as in_file:
    for line in in_file:
        split_line = line.split('\t')    # The input file is in tab delimited format, so we split by tab.
        chromosome = split_line[0]       # The 1st column of the file contains the chromosome info.
        feature = split_line[2]          # the 3rd column contains the feature info.
        
        # The elements of the list returned by .split() are string, so to get numerical values for
        # "start" and "end" we must convert them to int().
        start = int(split_line[3]) 
        end   = int(split_line[4]) 
        is_forward_strand = split_line[6] == '+'    # The 7th column contains the strand info: + = forward; - = reverse.
    
        if chromosome == 'MT' and feature == 'gene' and is_forward_strand and start > 7500 and end < 10000:
            to_report.append(line)

for line in to_report:
    print(line, end='')

<br>

## Exercise 3.3
Write the sequences of every exon of the mitochondrial chromosome between coordinates 7500 and 12500 to a file, in the <a href="https://en.wikipedia.org/wiki/FASTA_format">fasta</a> format.  
In the output fasta file, identify the exons using the name of their gene, as well as their start and end positions.

For instance, for an exon of gene ENSG000002, going from position 42 to 1337 : `>ENSG000002:42-1337`

Use the same GFF file you used in the previous exercise, 
as well as the fasta file `Homo_sapiens.GRCh38.dna.chromosome.MT.fa`, which contains the sequence of the mitochondrial chromosome..


In [3]:

# Let's first create a function to extract the gene name for the "Additional Info" field of a GFF file. 
# This will simplify our code later.
def get_gene_name_from_additional_info(field):
    """Extracts the gene name from the 'additionnal info' field of a GFF file"""
    subfields = field.split(';')   # The "additional info" field uses ';' as sub-field delimiter.
    #print(split_field)
    gene_name = ''
    
    # Looking for a pattern like 'gene_id "ENSG00000198786"'
    for subfield in subfields:
        words = subfield.split(' ')
        if words[0] == "gene_id": 
            gene_name = words[1].strip('"')  # Remove the quote around subfield value.
            break                            # if the gene_id subfield is found, exit loop immediatly.
            
    return gene_name
    

# Part 1: extract the gene name and coordinate positions of the exons of interest from the GTF file.
desired_chromosome = 'MT'
desired_feature = 'exon'
extraction_begin = 7500
extraction_end = 12500

# Create an empty list to store 
exons = []
with open("../exercises/Homo_sapiens.GRCh38.99.MT.gtf" , 'r') as in_file:
    for line in in_file:
        # GFF is a tab delimited format, we split the input line by tab.
        split_line = line.split('\t')
        
        # Retrieve info from the different fields of the input line.
        chromosome = split_line[0]
        feature = split_line[2]
        start = int(split_line[3]) 
        end   = int(split_line[4]) 
        is_forward_strand = split_line[6] == '+'   # The 7th column contains the strand info: + = forward; - = reverse.
       
        if chromosome == desired_chromosome and feature == desired_feature and start >= extraction_begin and end <= extraction_end:
            # Extract the gene name form the additional info field - the 9th column of the file.
            gene_name = get_gene_name_from_additional_info(split_line[8])
            exons.append((gene_name, start, end))


# Part 2: extract the whole sequence of the mitochondrial chromosome from the FASTA file.
with open( "../exercises/Homo_sapiens.GRCh38.dna.chromosome.MT.fa", 'r' ) as in_file:
    # Read the header line, as it does not contain nucleotides but only the FASTA header. 
    l = in_file.readline()
    
    # Loop through the remainer of the file to extract the entire nucleotide sequence of the MT chromosome.
    sequence = ''
    for l in in_file:
        sequence += l.strip()
    
# Part 3: write exons and their sequences in FASTA format.
with open("sequences.fa", 'w') as out_file:
    for exon_name, start, end in exons:   # Note: when iterating over a list of tuples that all have the same size, 
                                          # the elements of each tuple can be directly unpacked into variables.
        # Here, we have to be careful: 
        #  * the GFF files indexing starts at 1, but python strings start indexing at 0, 
        #    so we have to add +1 to start and end GFF positions.
        #  * the slicing excludes the end position, therfore we have to add another + 1
        #    to the end value.
        exon_seq = sequence[start + 1: end + 1 + 1 ]

        print('>', exon_name, ':', start, '-', end, sep='', file=out_file)
        print(exon_seq, file=out_file)

